In [2]:
import configs
import strava
import requests

import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-pastel')



In [3]:
resp = strava.auth().json()
access_token = resp['access_token']
activity_id = 7169676460

In [5]:
# returns: zones, laps, and streams
# I invision that these will become three tables adjacent to the existing "activities" table
# The three new tables branch from the activities table, where id is primary key for all

data = strava.get_activity_details(access_token, activity_id)
data

{'zones': [{'score': 42.0,
   'distribution_buckets': [{'min': 0, 'max': 123, 'time': 30},
    {'min': 123, 'max': 153, 'time': 908},
    {'min': 153, 'max': 169, 'time': 553},
    {'min': 169, 'max': 184, 'time': 147},
    {'min': 184, 'max': -1, 'time': 14}],
   'type': 'heartrate',
   'resource_state': 3,
   'sensor_based': True,
   'points': 4,
   'custom_zones': False}],
 'laps': [{'id': 23751689135,
   'resource_state': 2,
   'name': 'Lap 1',
   'activity': {'id': 7169676460, 'resource_state': 1},
   'athlete': {'id': 98390356, 'resource_state': 1},
   'elapsed_time': 550,
   'moving_time': 550,
   'start_date': '2022-05-19T16:09:04Z',
   'start_date_local': '2022-05-19T12:09:04Z',
   'distance': 1609.34,
   'start_index': 0,
   'end_index': 153,
   'total_elevation_gain': 4.4,
   'average_speed': 2.93,
   'max_speed': 4.539,
   'average_cadence': 88.5,
   'device_watts': False,
   'average_heartrate': 147.7,
   'max_heartrate': 166.0,
   'lap_index': 1,
   'split': 1,
   'pace_z